In [ ]:
import cv2
from PIL import Image
import imageio
from tqdm import tqdm

import torch
import torch.nn.init
from torch.utils.data import DataLoader
from torchvision import transforms
import torchsummary

import json
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import time
import os

import util
import NeuralNets as NN
from dataset import StUnstDataset
import random

LEARNING_RATE =  0.01
EPOCHS = 128
BATCH_SIZE = 2

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

TRAIN_PATH = "./train_data/img/"
LABEL_PATH = "./train_data/label.json"

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)

if device == 'cuda':
    torch.cuda.manual_seed_all(777)


In [ ]:

def showData(Dataset, model1=None):
    fig, ax = plt.subplots(3, 5)

    for i in range(5):
        randId = int(random.random() * len(Dataset))
        trainData = Dataset[randId]
        img = trainData[0]

        if model1== None:
            y = trainData[1]

        else:
            model.eval()
            out = NN.run_model_img(img, model, device)
            



        ax[0, i].imshow(heatImg)
        ax[1, i].imshow(out1, cmap='gray')
        ax[2, i].imshow(out2, cmap='gray')
        ax[0, i].set_title("train {}".format(i))

    plt.tight_layout()
    plt.show()

In [ ]:
trainDataset = SegDataset(TRAIN_PATH, LABEL_PATH, limit=40)
train_loader =  DataLoader(dataset = trainDataset, batch_size=BATCH_SIZE, shuffle=True)

showData(trainDataset)

model = NN.SegModel()
model = torch.nn.DataParallel(model)
model.to(device)

import torchsummary
torchsummary.torchsummary.summary(model, batch_size=BATCH_SIZE,device=device,input_size=(3, 480, 640))

In [ ]:
criterion = torch.nn.MSELoss().to(device) 
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
train_losses = []

for singleEpoch in range(1, EPOCHS+1):
    train_loss = 0.0
    total = 0
    model.train()

    for data, target in tqdm(train_loader, desc="Epoch: {}".format(singleEpoch)):
        data = data.permute(0, 3, 1, 2)
        data = data.cuda().float()
        
        target = target.cuda().float()


        # cleaer the gradients all optimized variables
        optimizer.zero_grad()

        # forward process
        out = model(data)[0]

        loss = criterion(out, target)
        loss.backward()
        
        optimizer.step()
        total += BATCH_SIZE
        train_loss += loss.item()

    # calculate average loss
    train_loss = train_loss/total
    train_losses.append(train_loss)


    print('Epoch: {} \tTraining Loss: {:.5f}'.format(singleEpoch, train_loss))

In [ ]:

# model = NN.load_model("./weights/model_1.pt", model)
NN.saveModel(model, "./weights/segmentModel.pt")